In [1]:
# tf tools
import tensorflow as tf
# image processsing
from tensorflow.keras.preprocessing.image import (load_img,
                                                  img_to_array,
                                                  ImageDataGenerator)
# VGG16 model
from tensorflow.keras.applications.vgg16 import (preprocess_input,
                                                 decode_predictions,
                                                 VGG16)
# layers
from tensorflow.keras.layers import (Flatten, 
                                     Dense, 
                                     Dropout, 
                                     BatchNormalization)
# generic model object
from tensorflow.keras.models import Model
# optimizers
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import SGD
#scikit-learn
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
# for plotting
import numpy as np
import matplotlib.pyplot as plt
import os
# for json files
import pandas as pd

2023-04-23 20:19:59.283007: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# plotting function
def plot_history(H, epochs):
    plt.style.use("seaborn-colorblind")

    plt.figure(figsize=(12,6))
    plt.subplot(1,2,1)
    plt.plot(np.arange(0, epochs), H.history["loss"], label="train_loss")
    plt.plot(np.arange(0, epochs), H.history["val_loss"], label="val_loss", linestyle=":")
    plt.title("Loss curve")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.tight_layout()
    plt.legend()

    plt.subplot(1,2,2)
    plt.plot(np.arange(0, epochs), H.history["accuracy"], label="train_acc")
    plt.plot(np.arange(0, epochs), H.history["val_accuracy"], label="val_acc", linestyle=":")
    plt.title("Accuracy curve")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.tight_layout()
    plt.legend()
    plt.show()

In [3]:
# Load the json metadata (containing the labels)
test_meta_path = os.path.join("..", "..", "..", "431824", "images", "metadata","test_data.json" )
test_metadata = pd.read_json(test_meta_path, lines=True)

train_meta_path = os.path.join("..", "..", "..", "431824", "images", "metadata","train_data.json" )
train_metadata = pd.read_json(train_meta_path, lines=True)

val_meta_path = os.path.join("..", "..", "..", "431824", "images", "metadata","val_data.json" )
val_metadata = pd.read_json(val_meta_path, lines=True)

In [4]:
# creating random samples to make machine faster
test_metadata = test_metadata.sample(n=75) #125
train_metadata = train_metadata.sample(n=250) #500
val_metadata = val_metadata.sample(n=50) #100

In [5]:
# generators for splitting data into test, train and validation
train_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
)
val_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
)
test_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
)

In [6]:
# Split the data and combining images and labels
directory_images = os.path.join("..", "..", "..", "431824")
TARGET_size=(224, 224)
BATCH_size = 32

#training data
train_images = train_generator.flow_from_dataframe(
    dataframe=train_metadata,
    directory = directory_images,
    x_col='image_path',
    y_col='class_label',
    target_size=TARGET_size,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_size,
    shuffle=True,
    seed=42,
    subset='training'
)
# validation data
val_images = val_generator.flow_from_dataframe(
    dataframe=val_metadata,
    directory = directory_images, 
    x_col='image_path',
    y_col='class_label',
    target_size=TARGET_size,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_size,
    shuffle=True,
    seed=42,
)
# test data
test_images = test_generator.flow_from_dataframe(
    dataframe=test_metadata,
    directory = directory_images,
    x_col='image_path',
    y_col='class_label',
    target_size=TARGET_size,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_size,
    shuffle=False
)

Found 250 validated image filenames belonging to 15 classes.
Found 50 validated image filenames belonging to 15 classes.
Found 75 validated image filenames belonging to 15 classes.


In [7]:
# load the VGG16 model without classifier layers
model = VGG16(include_top=False,
              pooling='avg', 
              input_shape=(224, 224, 3)) # set to fit the size of the indo images
# mark loaded layers as not trainable (freeze all weights)
for layer in model.layers: 
    layer.trainable = False

In [8]:
# add new classifier layers
flat1 = Flatten()(model.layers[-1].output) # flatten the images
bn = BatchNormalization()(flat1) # batch normalization layer
class1 = Dense(256, 
               activation='relu')(bn)
class2 = Dense(128, 
               activation='relu')(class1)
output = Dense(10, 
               activation='softmax')(class2)

# define new model
model = Model(inputs=model.inputs, 
              outputs=output)

In [9]:
# compile 
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=10000,
    decay_rate=0.9)
sgd = SGD(learning_rate=lr_schedule)
model.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# summarize model
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [10]:
# fit model to indo fashion and train model
history = model.fit(
    train_images, 
    steps_per_epoch = len(train_images),
    validation_data  = val_images,
    validation_steps = len(val_images),
    epochs = 10
    )

Epoch 1/10


2023-04-23 20:20:08.707306: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-04-23 20:20:14.311417: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: logits and labels must be broadcastable: logits_size=[32,10] labels_size=[32,15]
	 [[{{node categorical_crossentropy/softmax_cross_entropy_with_logits}}]]


InvalidArgumentError: Graph execution error:

Detected at node 'categorical_crossentropy/softmax_cross_entropy_with_logits' defined at (most recent call last):
    File "/usr/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/coder/.local/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/coder/.local/lib/python3.9/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/home/coder/.local/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/home/coder/.local/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.9/asyncio/base_events.py", line 596, in run_forever
  